In [ ]:
#plan

#break soup into sections
#break sections into wrappers
#define functions to section titles, get data headers, data cells
#make a dict with {data-header : data-cells} for each wrapper
    #add the {'Hunt Title' : section title} to the dict
#get a list of dicts, one for each secion
#create dataframe
#load to csv!



#still need to figure out:
#why is dataframe duplicating records?
#how to get the Last Year data cell broken down into three key-value pairs in the dictionary
#how to fully clean text
#data types?


In [68]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

#page = requests.get('https://tpwd.texas.gov/huntwild/hunt/public/public_hunt_drawing/hunt-category-details.phtml?OCat=GDA')
#page = requests.get('https://tpwd.texas.gov/huntwild/hunt/public/public_hunt_drawing/hunt-category-details.phtml?OCat=GDE')
#page = requests.get('https://tpwd.texas.gov/huntwild/hunt/public/public_hunt_drawing/hunt-category-details.phtml?OCat=GFH')
page = requests.get('https://tpwd.texas.gov/huntwild/hunt/public/public_hunt_drawing/hunt-category-details.phtml?OCat=MDG')
soup = BeautifulSoup(page.content, 'html.parser')
sections = soup.find_all('section')


def get_section_title(soup_obj):
        title_str = soup_obj.find('span',class_='title').get_text()
        return title_str

def get_data_wrapper(section):
    data_wrappers = section.select('div.data-wrapper')
    return data_wrappers

def get_data_header_text(wrapper):
    data_header_text = wrapper.select('div.data-header')[0].get_text()
    return data_header_text

def get_data_value_text(wrapper):
    data_value_text = wrapper.select('div.data-cell')[0].get_text(strip=True).replace("\r\n","")
    return data_value_text

def get_data_row(soup_obj):
    data_row = soup_obj.find_all('div',class_='data-cell')
    return data_row




In [69]:
#Generate dataframe for fields with one data-cell per data-row

list_of_dicts = []
for section in sections:
    data_wrappers = get_data_wrapper(section)
    d = {}
    d['Title'] = get_section_title(section)
    for wrapper in data_wrappers:
        for cell in data_wrappers:
            d[get_data_header_text(wrapper)] = get_data_value_text(wrapper)
    list_of_dicts.append(d)


df1 = pd.DataFrame.from_records(list_of_dicts)

In [70]:
# generate dataframe for Permits, Age Requirements, and Last Year, which have multiple data-cells per row

def split_data_cell(data_cell):
    pair = data_cell.split(':')
    return pair

keywords = ['Available', 'Fee', 'Youth ages', 'Adult minimum', 'Supervising adult', 'Applicants', 'Permits/Groups', 'Success Rate']


list_of_dicts1 = []
for section in sections:
    data_wrappers = get_data_wrapper(section)
    d = {}
    for wrapper in data_wrappers:
            data_row = get_data_row(wrapper)
            for cell in data_row:
                for i in keywords:
                    if i in cell.get_text():
                        split_values = cell.get_text(strip=True).split(':')
                        key = split_values[0].replace("\r\n","")
                        value = split_values[1].replace("\r\n","")
                        d[key] = value
                    else:
                        continue
    list_of_dicts1.append(d)
    
df2 = pd.DataFrame.from_records(list_of_dicts1)
df2

,Available,Fee per adult,Fee per youth,Youth ages,Adult minimum age,Supervising adult minimum age,Applicants,Permits/Groups,Success Rate
0,12,$80.00,$0.00,8- 16,17,18,6108,,
1,8,$130.00,$0.00,8- 16,17,18,5818,,
2,8,$130.00,$0.00,8- 16,17,18,4798,8,13%
3,4,$130.00,$0.00,8- 16,17,18,4383,,
4,10,$130.00,$0.00,8- 16,17,18,5071,10,45%
5,2,$130.00,$0.00,8- 16,17,18,5328,,
6,1,$130.00,$0.00,8- 16,17,18,3132,,


In [71]:
#join the two dataframes
df3 = pd.concat([df1.reset_index(drop=True),df2.reset_index(drop=True)], axis=1)
df3


#clean fields
df3['Hunt Dates'] = df3['Hunt Dates'].str.replace(' ','')
df3['Hunt Dates'] = df3['Hunt Dates'].str.replace('–','; ')
df3['Youth ages'] = df3['Youth ages'].str.replace(' ','')
df3['Youth ages'] = df3['Youth ages'].str.replace('-',' to ')

#drop fields
try:
    df3 = df3.drop(columns=['Groups', 'Last Year', 'Age Requirements', 'Permits'])
except:
    pass

#add fields
#air_quality["london_mg_per_cubic"] = air_quality["station_london"] * 1.882
df3['Available'] = df3['Available'].astype(int)
df3['Applicants'] = df3['Applicants'].astype(float)
df3["Draw Probability"] = (df3["Available"] / df3["Applicants"])

df3


,Title,Hunt Dates,Bag Limit,ONLY MEANS ALLOWED,Hunt Restrictions,Hunt Method,Baiting,Permits,Age Requirements,Last Year,...,Available,Fee per adult,Fee per youth,Youth ages,Adult minimum age,Supervising adult minimum age,Applicants,Permits/Groups,Success Rate,Draw Probability
0,Big Bend Ranch SP,"Dec06,2021; Dec09,2021",One Mule Deer; Buck OnlyOne Javelina; Either S...,Centerfire Rifles .243 or Larger,Any vehicle with an elevated seat or shooting ...,Compartment; It is strongly recommended that h...,Not allowed,Available: 12,Youth ages: 8- 16,Applicants: 6108,...,12,$80.00,$0.00,8 to 16,17,18,6108.0,,,0.001965
1,Black Gap WMA,"Dec27,2021; Dec30,2021",One White-tailed Deer or Mule Deer; Buck OnlyU...,NaN,NaN,Compartment,Not allowed,Available: 8,Youth ages: 8- 16,Applicants: 5818,...,8,$130.00,$0.00,8 to 16,17,18,5818.0,,,0.001375
2,Caprock Canyons SP,"Dec07,2021; Dec10,2021",One White-tailed Deer or Mule Deer; Either Sex...,Centerfire Rifles,Camping is available for hunters only. Vehicul...,"Compartment; Also, the removal of the harveste...",Not allowed,Available: 8,Youth ages: 8- 16,Applicants: 4798,...,8,$130.00,$0.00,8 to 16,17,18,4798.0,8,13%,0.001667
3,Elephant Mountain WMA,"Dec27,2021; Dec30,2021",One White-tailed Deer or Mule Deer; Buck Only;...,NaN,NaN,Compartment,Not allowed,Available: 4,Youth ages: 8- 16,Applicants: 4383,...,4,$130.00,$0.00,8 to 16,17,18,4383.0,,,0.000913
4,Matador WMA,"Dec06,2021; Dec10,2021",Two Deer; Either Sex; Limit One Buck; Buck mus...,NaN,NaN,Compartment,Not allowed,Available: 10,Youth ages: 8- 16,Applicants: 5071,...,10,$130.00,$0.00,8 to 16,17,18,5071.0,10,45%,0.001972
5,Sierra Diablo WMA,"Jan03,2022; Jan06,2022",One Mule Deer; Buck OnlyOne Elk; Either SexUnl...,Centerfire Rifles,NaN,Compartment; A four wheel drive vehicle is REQ...,Not allowed,Available: 2,Youth ages: 8- 16,Applicants: 5328,...,2,$130.00,$0.00,8 to 16,17,18,5328.0,,,0.000375
6,Yoakum Dunes WMA,"Dec06,2021; Dec10,2021",One Mule Deer; Buck Only; Limit One Buck; Matu...,Centerfire Rifles .243 or Larger,NaN,Guided,Not allowed,Available: 1,Youth ages: 8- 16,Applicants: 3132,...,1,$130.00,$0.00,8 to 16,17,18,3132.0,,,0.000319


In [72]:
#generate csv
df3.to_csv('hunt_data.csv')